# Stimare Q function ottimale
## 1. Quanto tempo ci mette per convergere a Q*, al variare di tau
## 2. Distanza tra Q* e Q_t 
## 3. distanza tra Q_0 e Q_0 appresa tramite i valori di tau
## 4. cercare di integrare i bound trovati

### Estrarre policy da Q
### Confrontare due diverse Q valutando diverse metriche 
### Calcolare Q*
### Tentativo di implementazione di un curriculum
### Plot functions

In [7]:
import numpy as np
from TMDP import TMDP
from river_swim import River

from algorithms import *


In [8]:
# Test with tau=0.9
nS = 8
gamma = 0.9
river = River(nS, gamma, 50, 1000)
tau = 0.9
xi = np.ones(river.nS)*1/river.nS
tmdp = TMDP(river, xi, tau, gamma)
tmdp_0 = TMDP(river, xi, 0, gamma)

In [9]:
s = tmdp.reset()
M = 1000
Q = np.zeros((tmdp.nS, tmdp.nA))
ret = 0
a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])
Q = Q_learning(tmdp, s, a, Q, M)

print("Q function with tau: {}\n".format(tmdp.tau))
print(Q)
print(get_policy(Q))

s = tmdp_0.reset()
M_0 = 100000
Q = np.zeros((tmdp_0.nS, tmdp_0.nA))
ret = 0
a = eps_greedy(s, Q, 1., tmdp_0.allowed_actions[s.item()])
Q = Q_learning(tmdp_0, s, a, Q, M_0)


print("Q function with tau: {}\n".format(tmdp_0.tau))
print(Q)
print(get_policy(Q))

r_s_a = compute_r_s_a(tmdp_0.nS, tmdp_0.nA, tmdp_0.P_mat, tmdp_0.reward)
q_star = bellman_optimal_q(tmdp_0.nS, tmdp_0.nA, tmdp_0.P_mat, tmdp_0.reward, 100000, gamma)


print("Q function with bellman operator\n")
print(q_star)
print(get_policy(q_star))


Q function with tau: 0.9

[[27.65858588  7.86464617]
 [ 9.7656388   6.15494918]
 [ 6.62316931 10.60300917]
 [10.01960772  6.33329262]
 [10.80757137  6.43258808]
 [ 6.28236037 10.39402085]
 [ 9.51543413  6.928138  ]
 [ 8.98908617  6.32903213]]
[0, 0, 1, 0, 0, 1, 0, 0]
Q function with tau: 0

[[500.         434.34353862]
 [450.         410.44097761]
 [405.         371.8386064 ]
 [364.5        333.53661187]
 [328.05       324.63096015]
 [  0.           0.        ]
 [  0.           0.        ]
 [  0.           0.        ]]
[0, 0, 0, 0, 0, 0, 0, 0]
Q function with bellman operator

[[500.         436.5       ]
 [450.         397.35      ]
 [405.         357.615     ]
 [364.5        325.61947683]
 [328.05       341.99806235]
 [307.79825612 461.16336549]
 [415.04702894 671.68637968]
 [604.51774171 990.63345095]]
[0, 0, 0, 0, 1, 1, 1, 1]


In [10]:

""" xi = np.ones(river.nS)*1/river.nS
gamma = 0.9
river = River(gamma)

for tau in taus:
    tmdp = TMDP(river, xi, tau, gamma)
    s = tmdp.reset()
    M = 10000
    Q = np.zeros((tmdp.nS, tmdp.nA))
    ret = 0
    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])
    Q = Q_learning(tmdp, s, a, Q, M)
    Qs.append({"tau":tau, "Q_function":Q, "env":tmdp})

for i in range(len(taus)):
    print("Tau:", Qs[i]['tau'])
    print(Qs[i]['Q_function']) """

' xi = np.ones(river.nS)*1/river.nS\ngamma = 0.9\nriver = River(gamma)\n\nfor tau in taus:\n    tmdp = TMDP(river, xi, tau, gamma)\n    s = tmdp.reset()\n    M = 10000\n    Q = np.zeros((tmdp.nS, tmdp.nA))\n    ret = 0\n    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])\n    Q = Q_learning(tmdp, s, a, Q, M)\n    Qs.append({"tau":tau, "Q_function":Q, "env":tmdp})\n\nfor i in range(len(taus)):\n    print("Tau:", Qs[i][\'tau\'])\n    print(Qs[i][\'Q_function\']) '

In [11]:
""" pi = get_policy(Qs[-2]['Q_function'])
pi_prime = get_policy(Qs[-1]['Q_function'])

print(pi)
print(pi_prime) """

" pi = get_policy(Qs[-2]['Q_function'])\npi_prime = get_policy(Qs[-1]['Q_function'])\n\nprint(pi)\nprint(pi_prime) "

In [15]:
taus = [1 - i*0.1 for i in range(10)]
taus.append(0)

# Curriculul for decreasing values of tau
for tau in taus:
    tmdp = TMDP(river, xi, tau, gamma)
    if tau == 1:
       Q = np.zeros((tmdp.nS, tmdp.nA))
    
    s = tmdp.reset()
    M = 500000
    ret = 0
    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])
    Q = Q_learning(tmdp, s, a, Q, M)


print("Q function learned with transfer learning:\n", Q)

pi = get_policy(Q)
print(pi)


Q function learned with transfer learning:
 [[500.         436.02273663]
 [450.         388.33373202]
 [405.         337.72713859]
 [364.5        322.89000421]
 [328.05       301.35897062]
 [295.2449986  268.10554084]
 [248.69454613 527.79363875]
 [392.79376009 820.48769454]]
[0, 0, 0, 0, 0, 0, 1, 1]
